## Imports

In [49]:
import numpy as np
import pandas as pd
import sklearn as sklearn
from sklearn.ensemble import HistGradientBoostingClassifier
print('Now using sklearn version '+sklearn.__version__)
print('Packages are Ready!!!')

Now using sklearn version 1.0.2
Packages are Ready!!!


## Loading data

In [50]:
def load_augmented_data(): #Loads the augmented dataset
    train_augmented=pd.read_csv(filepath_or_buffer='/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/train_augmented.csv')
    test_augmented=pd.read_csv(filepath_or_buffer='/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/test_augmented.csv')
    print('Augmented datasets are ready')
    return train_augmented , test_augmented

def load_original_data(): # Loads the original datasets of the competition
    test=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/test.csv')
    train=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/train.csv')
    print('Original datasets are ready')
    return train, test

In [51]:
train_augmented , test_augmented = load_augmented_data()

Augmented datasets are ready


## Resampling
We are going to resample the missrepresented class in the training dataset, in this way we are going to balance out the classes.

In [4]:
training_set=x_train[x_train.columns[1:]].to_numpy()
training_labels=train['target'].to_numpy()


array([[ 8.9255, -6.7863, 11.9081, ...,  0.    ,  0.    ,  0.    ],
       [11.5006, -4.1473, 13.8588, ...,  0.    ,  0.    ,  0.    ],
       [ 8.6093, -2.7457, 12.0805, ...,  0.    ,  0.    ,  0.    ],
       ...,
       [11.2232, -5.0518, 10.5127, ...,  0.    ,  0.    ,  1.    ],
       [ 9.7148, -8.6098, 13.6104, ...,  0.    ,  0.    ,  1.    ],
       [10.8762, -5.7105, 12.1183, ...,  0.    ,  0.    ,  1.    ]])

In [58]:
x_train


,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_unique?,var_191_unique?,var_192_unique?,var_193_unique?,var_194_unique?,var_195_unique?,var_196_unique?,var_197_unique?,var_198_unique?,var_199_unique?
0,train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,0,0,0,0,0,0,0,0,0,0
1,train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,0,0,0,0,0,0,0,0,0,0
2,train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,0,0,0,0,0,0,0,0,0,0
3,train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,0,0,0,0,0,0,0,0,0,0
4,train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,train_199995,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,-0.6209,...,0,1,0,0,0,0,1,0,0,0
199996,train_199996,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,-3.6797,...,0,0,0,1,0,0,0,0,0,0
199997,train_199997,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,0.1202,...,0,0,0,0,0,0,0,0,0,1
199998,train_199998,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,-2.1926,...,0,0,0,0,0,0,0,0,0,1


In [57]:
train_augmented

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190_unique?,var_191_unique?,var_192_unique?,var_193_unique?,var_194_unique?,var_195_unique?,var_196_unique?,var_197_unique?,var_198_unique?,var_199_unique?
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,0,0,0,0,0,0,0,0,0,0
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,0,0,0,0,0,0,0,0,0,0
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,0,0,0,0,0,0,0,0,0,0
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,0,0,0,0,0,0,0,0,0,0
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,0,1,0,0,0,0,1,0,0,0
199996,train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,0,0,0,1,0,0,0,0,0,0
199997,train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,0,0,0,0,0,0,0,0,0,1
199998,train_199998,0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,0,0,0,0,0,0,0,0,0,1


In [65]:

"""Since we are dealing with an imbalanced data set we are going to deal with it  by oversampling the underrepresented class"""

from imblearn.over_sampling import RandomOverSampler
# we define a function that are going to balance the classes in a training set
def resample_data(dataset): #gets a dataset and returns the balanced version of them
    rov=RandomOverSampler()
    x_resampled, y_resampled = rov.fit_resample(
        X=dataset[dataset.columns[2:]].to_numpy(),
        y=dataset['target'].to_numpy())
    return x_resampled, y_resampled # return the dataset and their targets


""" Let's use the function that we just declared"""
x_resampled, y_resampled = resample_data(train_augmented)
""" We can now see the number example for each class"""
pd.Series(name='Number of examples', data=y_resampled).value_counts()

0    179902
1    179902
Name: Number of examples, dtype: int64

In [12]:
"""initializes an instance of a gradient boosting classifier """
def get_hgbc():
    hgbc=sklearn.ensemble.HistGradientBoostingClassifier(
        #Hyper parameters
        max_iter=20, #Trees to be build
        early_stopping=True,
        #validation_fraction=0.02,
        learning_rate=0.1,
        warm_start=True,
        #Metrics and over-fitting
        loss='binary_crossentropy',
        l2_regularization=0.01,
        scoring='roc_auc',
        # Tree parameters
        min_samples_leaf=120,
        max_leaf_nodes=13,
        max_depth=None,
        #ETC
        verbose=0
    )
    return hgbc
model=get_hgbc()

## CROSS VALIDATION SCHEME TRAINING

In [13]:

""" TEST DO NOT TOUCH"""

from sklearn.model_selection import cross_val_score
val_scores=cross_val_score(model, x_resampled, y_resampled, cv=5, scoring='roc_auc')


In [14]:
val_scores

array([0.80078877, 0.80131791, 0.79696717, 0.79688244, 0.79710308])

In [15]:
"""Creates and fits model with the resampled data"""
model.fit(x_resampled, y_resampled, sample_weight=None)
print('Training done!')


Training done!


## Getting predictions

In [17]:
"""Let's make predictions with the model"""
super_test_set=x_test[x_test.columns[1:]].to_numpy()
predictions=model.predict(super_test_set)
#TEST
predictions_prob=model.predict_proba(super_test_set)
predictions_prob=pd.DataFrame(predictions_prob)
######
predictions_prob
super_test_set

array([[ 8.5374, -1.3222, 12.022 , ...,  0.    ,  0.    ,  1.    ],
       [17.3035, -2.4212, 13.3989, ...,  0.    ,  1.    ,  0.    ],
       [10.6137, -2.1898,  8.909 , ...,  0.    ,  0.    ,  1.    ],
       ...,
       [ 8.2964, -2.3119, 11.2139, ...,  0.    ,  0.    ,  0.    ],
       [11.636 ,  2.2769, 11.2074, ...,  0.    ,  0.    ,  0.    ],
       [13.5745, -0.5134, 13.6584, ...,  0.    ,  0.    ,  0.    ]])

In [48]:
""" We create a fucntion that uses the trained model to save a CVS file ready for sumbmission"""
def get_sumbmission(model):

    return 0


"""Let's get ready for submission. Creating a DataFrame containing ID codes and targets"""
submission_df=pd.DataFrame()
submission_df['ID_code']=x_test['ID_code']

#TEST
#submission_df['target']=predictions THIS IS THE REAL DEAL
submission_df['target']=predictions_prob[1].values #TEST
########################

"""we are also creating a dummy columns to help us sort our data by ID code"""
submission_df['dummy_index']=submission_df['ID_code'].apply(lambda x: int(x[5:]))

submission_df

"""Let's sort our data and save it as a CSV file! We are done!"""
juan_submission=submission_df.sort_values(by='dummy_index')
juan_submission.drop(['dummy_index'],axis=1).to_csv('juan_submission.csv', index=False)
final_submission=pd.read_csv(filepath_or_buffer='juan_submission.csv')

In [52]:
!ls

Boosting copy.ipynb       juan_submission.csv       x_test.csv
DataGeneration copy.ipynb test.csv                  x_train.csv
README.md                 train.csv


In [51]:
final_submission

,ID_code,target
0,test_0,0.503196
1,test_1,0.791701
2,test_2,0.769796
3,test_3,0.487299
4,test_4,0.378554
...,...,...
199995,test_199995,0.320715
199996,test_199996,0.065481
199997,test_199997,0.104253
199998,test_199998,0.472951


Boosting copy.ipynb       juan_submission.csv       x_test.csv
DataGeneration copy.ipynb test.csv                  x_train.csv
README.md                 train.csv
